
# Fine-tuning T5 Summarization

---

In [ ]:
!nvidia-smi

Wed Mar 23 06:03:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Setup

---

In [ ]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [ ]:
! pip install transformers
! pip install datasets
! pip install sentencepiece
! pip install rouge_score
! pip install wandb

     |████████████████████████████████| 3.8 MB 9.1 MB/s 
     |████████████████████████████████| 596 kB 47.9 MB/s 
     |████████████████████████████████| 67 kB 4.1 MB/s 
     |████████████████████████████████| 6.5 MB 40.7 MB/s 
     |████████████████████████████████| 895 kB 46.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 6.4 MB/s 
     |████████████████████████████████| 134 kB 33.6 MB/s 
     |████████████████████████████████| 212 kB 36.7 MB/s 
     |████████████████████████████████| 1.1 MB 33.3 MB/s 
     |████████████████████████████████| 127 kB 30.6 MB/s 
     |████████████████████████████████| 144 kB 37.5 MB/s 
     |████████████████████████████████| 271 kB 38.6 MB/s 
     |████████████████████████████████| 94 kB 1.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [ ]:
import torch
import numpy as np
import datasets

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

from tabulate import tabulate
import nltk
from datetime import datetime

In [ ]:
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
    import wandb

    wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Model and tokenizer

---

Hiperparámetros: 

[HF Bart configuration](https://huggingface.co/transformers/_modules/transformers/configuration_bart.html)

[Fairseq Bart](https://github.com/pytorch/fairseq/tree/master/examples/bart)

In [ ]:
import tensorflow as tf
print(tf.test.is_built_with_cuda())

True


In [ ]:
#Llamado del modelo
model_name = "t5-small"
#Definición de modelo y tokenizador
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Se fijan los parámetros del modelo
model.config.activation_dropout = 0.0
model.config.classif_dropout = 0.0
model.config.max_encoder_position_embeddings = 1024
model.config.max_decoder_position_embeddings = 16384
model.config.min_length = 350
model.config.max_length = 500
print(model.config)

# tokenización
encoder_max_length = 256 
decoder_max_length = 64

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

T5Config {
  "_name_or_path": "t5-small",
  "activation_dropout": 0.0,
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "classif_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "max_decoder_position_embeddings": 16384,
  "max_encoder_position_embeddings": 1024,
  "max_length": 500,
  "min_length": 350,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "

## Data

---

In [ ]:
#Lista de datasets compatibles con la biblioteca
# datasets.list_datasets()

### Descarga y Preparación de los Datos

### Cargado de Dataset

In [ ]:
train_data_txt = datasets.load_dataset("billsum", '3.0.0', split="train")
validation_data_txt = datasets.load_dataset("billsum", '3.0.0', split="test")

Using custom data configuration 3.0.0


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating ca_test split: 0 examples [00:00, ? examples/s]

Dataset billsum downloaded and prepared to /root/.cache/huggingface/datasets/billsum/3.0.0/3.0.0/d1e95173aed3acb71327864be74ead49b578522e4c7206048b2f2e5351b57959. Subsequent calls will reuse this data.


Using custom data configuration 3.0.0
Reusing dataset billsum (/root/.cache/huggingface/datasets/billsum/3.0.0/3.0.0/d1e95173aed3acb71327864be74ead49b578522e4c7206048b2f2e5351b57959)


**Preprocess and tokenize**

In [ ]:
def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    source, target = batch["text"], batch["summary"]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch


train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
)

validation_data = validation_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=validation_data_txt.column_names,
)

  0%|          | 0/19 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

## Training

---

### Metrics

In [ ]:
# Borrowed from https://github.com/huggingface/transformers/blob/master/examples/seq2seq/run_summarization.py

nltk.download("punkt", quiet=True)

metric = datasets.load_metric("rouge")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

### Training arguments

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=10,  # Deseado: 10
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=64,  # numeros al poder de 2, 64 o 32 son buenos valores, intentar ambos
    per_device_eval_batch_size=64,
    learning_rate=3e-04, #3e-05 puede ser tambien
    warmup_steps=50,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True, #Para métricas ROUGE
    logging_dir="logs",
    logging_steps=50,
    save_total_limit=3, #guardar ultimos 3 checkpoints
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


### Train

Wandb integration

In [ ]:
wandb

<module 'wandb' from '/usr/local/lib/python3.7/dist-packages/wandb/__init__.py'>

In [ ]:
if WANDB_INTEGRATION:
    wandb_run = wandb.init(
        project="T5_BILLSUM_FT",
        config={
            "per_device_train_batch_size": training_args.per_device_train_batch_size,
            "learning_rate": training_args.learning_rate,
            "dataset": "billsum",
        },
    )

    now = datetime.now()
    current_time = now.strftime("%H%M%S")
    wandb_run.name = "run_" + current_time

Evaluate before fine-tuning

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 3269
  Batch size = 64


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


{'eval_gen_len': 401.9716,
 'eval_loss': 5.112339496612549,
 'eval_rouge1': 30.3339,
 'eval_rouge2': 12.5374,
 'eval_rougeL': 22.0445,
 'eval_rougeLsum': 23.4351,
 'eval_runtime': 384.7183,
 'eval_samples_per_second': 8.497,
 'eval_steps_per_second': 0.135}

Train the model

In [ ]:
torch.cuda.is_available()
import tensorflow as tf

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
#%%wandb
# uncomment to display Wandb charts

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 18949
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 2970


Step,Training Loss
50,4.872700
100,4.224900
150,3.902500
200,3.766400
250,3.629600
300,3.525700
350,3.507400
400,3.427400
450,3.381300
500,3.344800


Saving model checkpoint to results/checkpoint-500
Configuration saved in results/checkpoint-500/config.json
Model weights saved in results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-500/tokenizer_config.json
Special tokens file saved in results/checkpoint-500/special_tokens_map.json
Copy vocab file to results/checkpoint-500/spiece.model
Saving model checkpoint to results/checkpoint-1000
Configuration saved in results/checkpoint-1000/config.json
Model weights saved in results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in results/checkpoint-1000/special_tokens_map.json
Copy vocab file to results/checkpoint-1000/spiece.model
Saving model checkpoint to results/checkpoint-1500
Configuration saved in results/checkpoint-1500/config.json
Model weights saved in results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1500/tokeni

TrainOutput(global_step=2970, training_loss=3.1519923020693588, metrics={'train_runtime': 2062.7099, 'train_samples_per_second': 91.865, 'train_steps_per_second': 1.44, 'total_flos': 1.282295898046464e+16, 'train_loss': 3.1519923020693588, 'epoch': 10.0})

Evaluate after fine-tuning

In [23]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 3269
  Batch size = 64


{'epoch': 10.0,
 'eval_gen_len': 350.0,
 'eval_loss': 2.897089719772339,
 'eval_rouge1': 44.7581,
 'eval_rouge2': 29.7323,
 'eval_rougeL': 38.4951,
 'eval_rougeLsum': 39.6863,
 'eval_runtime': 307.1198,
 'eval_samples_per_second': 10.644,
 'eval_steps_per_second': 0.169}

In [24]:
if WANDB_INTEGRATION:
    wandb_run.finish()

eval/gen_len,█▁
eval/loss,█▁
eval/rouge1,▁█
eval/rouge2,▁█
eval/rougeL,▁█
eval/rougeLsum,▁█
eval/runtime,█▁
eval/samples_per_second,▁█
eval/steps_per_second,▁█
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████


## Evaluation

---

**Generate summaries from the fine-tuned model and compare them with those generated from the original, pre-trained one.**

In [25]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["text"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)

test_samples = validation_data_txt.select(range(16))

summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_summary(test_samples, model)[1]

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
 

In [29]:
print(
    tabulate(
        zip(
            range(len(summaries_after_tuning)),
            summaries_after_tuning,
            summaries_before_tuning,
        ),
        headers=["Id", "Summary after", "Summary before"],
    )
)
print("\nTarget summaries:\n")
print(
    tabulate(list(enumerate(test_samples["summary"])), headers=["Id", "Target summary"])
)
# print("\nSource documents:\n")
# print(tabulate(list(enumerate(test_samples["text"])), headers=["Id", "Document"]))

  Id  Summary after                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [30]:
!zip -r /content/file.zip /content/results/checkpoint-2500/ #VER CHECKPOINT

  adding: content/results/checkpoint-2500/ (stored 0%)
  adding: content/results/checkpoint-2500/tokenizer_config.json (deflated 80%)
  adding: content/results/checkpoint-2500/spiece.model (deflated 48%)
  adding: content/results/checkpoint-2500/rng_state.pth (deflated 27%)
  adding: content/results/checkpoint-2500/scheduler.pt (deflated 49%)
  adding: content/results/checkpoint-2500/config.json (deflated 63%)
  adding: content/results/checkpoint-2500/tokenizer.json (deflated 74%)
  adding: content/results/checkpoint-2500/pytorch_model.bin (deflated 7%)
  adding: content/results/checkpoint-2500/special_tokens_map.json (deflated 83%)
  adding: content/results/checkpoint-2500/optimizer.pt (deflated 7%)
  adding: content/results/checkpoint-2500/training_args.bin (deflated 49%)
  adding: content/results/checkpoint-2500/trainer_state.json (deflated 82%)


In [31]:
from google.colab import files
files.download("/content/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()